# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv # noqa

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust-tsilma,65.4410,52.1498,-9.10,96,92,1.79,RU,1698794366
1,1,ushuaia,-54.8000,-68.3000,6.79,64,40,10.73,AR,1698794211
2,2,georgetown,5.4112,100.3354,26.96,95,20,1.54,MY,1698794148
3,3,mariehamn,60.0973,19.9348,4.34,93,100,5.66,AX,1698794463
4,4,tchintabaraden,15.8969,5.7985,28.14,15,100,2.76,NE,1698794465


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', # x-axis: Longitude
    'Lat', # y-axis: Latitude
    geo=True, # To ensure the data is plotted on a map
    tiles='OSM', # Using OpenStreetMap as the tile source
    size='Humidity', # Size of the point will be based on the humidity value
    hover_cols=['City', 'Humidity'], # On hover, display the city name and its humidity
    color='Humidity', # Color of the point will be based on the humidity value
    colormap='viridis', # Using 'viridis' colormap
    title='City Locations by Humidity'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 10) & (city_data_df["Max Temp"] <= 35) & 
    (city_data_df["Wind Speed"] < 20) & 
    (city_data_df["Cloudiness"] < 50) & 
    (city_data_df["Humidity"] >= 20) & (city_data_df["Humidity"] <= 80)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,antofagasta,-23.6500,-70.4000,16.60,75,38,2.67,CL,1698794470
16,16,uturoa,-16.7333,-151.4333,26.30,78,5,9.75,PF,1698794325
20,20,haiku-pauwela,20.9219,-156.3051,29.01,58,40,4.12,US,1698794479
30,30,crane,31.3974,-102.3501,12.66,23,0,3.36,US,1698794485
32,32,pisco,-13.7000,-76.2167,20.03,77,0,9.77,PE,1698794486


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
7,antofagasta,CL,-23.6500,-70.4000,75,
16,uturoa,PF,-16.7333,-151.4333,78,
20,haiku-pauwela,US,20.9219,-156.3051,58,
30,crane,US,31.3974,-102.3501,23,
32,pisco,PE,-13.7000,-76.2167,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "types": "accommodation", 
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"circle:{lng},{lat},{radius}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
antofagasta - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
nacozari viejo - nearest hotel: No hotel found
valparaiso - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
kamaishi - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
geraldton - nearest hotel: No hotel found
mukalla - nearest hotel: No hotel found
redencao - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
la mira - nearest hotel: No hotel found
bam - nearest hotel: No hotel found
chonchi - nearest hotel: No hotel found
bubaque - nearest hotel: No hotel found
yeppoon - nearest hotel: No hotel found
spanish wells - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
7,antofagasta,CL,-23.6500,-70.4000,75,No hotel found
16,uturoa,PF,-16.7333,-151.4333,78,No hotel found
20,haiku-pauwela,US,20.9219,-156.3051,58,No hotel found
30,crane,US,31.3974,-102.3501,23,No hotel found
32,pisco,PE,-13.7000,-76.2167,77,No hotel found
...,...,...,...,...,...,...
558,oodweyne,SO,9.4092,45.0640,64,No hotel found
559,satkhira,BD,22.7167,89.1000,72,No hotel found
560,guerrero negro,MX,27.9769,-114.0611,26,No hotel found
567,haimen,CN,23.1935,116.6142,79,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', # x-axis: Longitude
    'Lat', # y-axis: Latitude
    geo=True, # To ensure the data is plotted on a map
    tiles='OSM', # Using OpenStreetMap as the tile source
    hover_cols=["Hotel Name", "Country"], 
    size="Humidity",  # Setting the size based on humidity, as per the previous steps
    color="Humidity", 
    colormap="viridis", 
    title="Hotel Locations based on Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    height=500,
    width=800
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)